## Home Assignment 1
## Finding Similar Items: Textually Similar Documents
### Group 11

Boyu Xue (boyux@kth.se), Kristupas Bajarunas(kristupasbajarunas@gmail.com)


### Description:

You are to implement the stages of finding textually similar documents based on Jaccard similarity using the shingling, minhashing, and locality-sensitive hashing (LSH) techniques and corresponding algorithms. The implementation can be done using any big data processing framework, such as Apache Spark, Apache Flink, or no framework, e.g., in Java, Python, etc. To test and evaluate your implementation, write a program that uses your implementation to find similar documents in a corpus of 5-10 or more documents such as web pages or emails.

The stages should be implemented as a collection of classes, modules, functions or procedures depending the framework and the language of your choice. Below, we give a description of sample classes that implement different stages of finding textually similar documents. You do not have to develop the exact same classes and data types as described below. Feel free to use data structures that suit you best.

* A class Shingling that constructs k–shingles of a given length k (e.g., 10) from a given document, computes a hash value for each unique shingle, and represents the document in the form of an ordered set of its hashed k-shingles.
* A class CompareSets that computes the Jaccard similarity of two sets of integers – two sets of hashed shingles.
* A class MinHashing that builds a minHash signature (in the form of a vector or a set) of a given length n from a given set of integers (a set of hashed shingles).
* A class CompareSignatures that estimates similarity of two integer vectors – minhash signatures – as a fraction of components, in which they agree.
* (Optional task for extra 2 bonus) A class LSH that implements the LSH technique: given a collection of minhash signatures (integer vectors) and a similarity threshold t, the LSH class (using banding and hashing) finds all candidate pairs of signatures that agree on at least fraction t of their components.

To test and evaluate scalability (the execution time versus the size of input dataset) of your implementation, write a program that uses your classes to find similar documents in a corpus of 5-10 documents. Choose a similarity threshold s (e.g., 0,8) that states that two documents are similar if the Jaccard similarity of their shingle sets is at least s. 

### Dataset

#### Health News in Twitter Data Set

#### Abstract: 
The data was collected in 2015 using Twitter API. This dataset contains health news from more than 15 major health news agencies such as BBC, CNN, and NYT.

Data Set Characteristics: Text

Number of Instances: 58000

Area: Computer

Attribute Characteristics: Real

Number of Attributes: 25000

Date Donated: 2018-02-19

#### Source:

Amir Karami (karami@sc.edu), University of South Carolina

https://archive.ics.uci.edu/ml/datasets/Health+News+in+Twitter

#### Data Set Information:

Each file is related to one Twitter account of a news agency. For example, bbchealth.txt is related to BBC health news. Each line contains tweet id|date and time|tweet. The separator is '|'. This text data has been used to evaluate the performance of topic models on short text data. However, it can be used for other tasks such as clustering.

### Implementation

#### 1. Import libraries and build-in functions

In [1]:
import random
import itertools
import os
import os.path
import glob
import numpy as np
import codecs
import time
from pprint import pprint
from collections import defaultdict
from functools import reduce
from collections import Counter

#### 2. Define the shingle function

This function is used to:
* Open files from a certain path (file_path).
* Convert the files to shingle sets with a length (k) of 5.
* Hash the shingles by using "hash" command of Python.
* Return the set of hashed shingles by using the "add" command to add value of hashed shingle to the sets of shingles.

In the file, each sentence includes a number and a date at the beginning, and a web address at the end. But we are not interested in the numbers, dates and web addresses. So we split the numbers, dates and web addresses by using "line.split" command. For example, in the sentence "586266687948881921|Thu Apr 09 20:37:25 +0000 2015|Drugs need careful monitoring for expiry dates, pharmacists say http://www.cbc.ca/news/health/drugs-need-careful-monitoring-for-expiry-dates-pharmacists-say-1.3026749?cmp=rss", only "Drugs need careful monitoring for expiry dates, pharmacists say" will be output to be hashed.


In [2]:
def hash_shingle(file_path,k=5):
    start_time = time.time()
    with open(file_path,'r',errors='ignore')as f:
        hash_shingle=set()
        for line in f:
            line=line.split('|')[2].split(' http')[0]
            for i in range(len(line)-k):
                hash_shingle.add(hash(line[i:i+k]))
    return hash_shingle

#### 3. Define the Jaccard similarity function

This function is used to:

* Calculate the intersection and union of two sets(s1 and s2).
* Divide the intersection by union and return the Jaccard similarity of the two sets.


In [3]:
def jacard_similarity(s1, s2):
    return len(s1.intersection(s2)) / len(s1.union(s2))

#### 4. Define the hash function

The hash function can be expressed as (a*value of shingle+b) mod c. The coefficients a and b will be random number less than the maximum value of x (2^32-1=4294967295). c is a prime number(4294967311) slightly bigger than the maximum value of x.

In [4]:
def hash_function(function_index, value):
    a = coefA[function_index]
    b = coefB[function_index]
    return (a * value + b) % nextPrime

#### 5. Define the get signature function

* For i in range of the number of hash function, calculate the returned value of hash function for the variable x (x will be the value of hash shingle from the sets of shingles)  by using the lambda function and the map function.

* Add the minimum value of the signature to the end of the signature list by using the append method.




In [5]:
def get_signature(set_of_shingles):
    signature = [] 
    for i in range(numH):
        signature.append(min(map((lambda x: hash_function(i,x) ), set_of_shingles)))
    return signature

#### 6. Define the locality sensitive hash (LSH) function

1. Record the running time by setting the start time and the end time.
2. Define finding similarity of LSH (find_sim_LSH) as a function of the number of bands(b) and the signature matrix. The number of rows(r) = number of hash functions/number of bands.
3. Set the candidate pairs as a list.
4. Build a dictionary named hash_table for the values of signatures by using the defaultdict command.
5. For each band and for every document(doc_id) select the rows of that band and document with current_band.
6. Add the band's signatures(current_band) to the bucket (current_hash) by using "hash" function of the tuple of signatures.
7. Store the buckets(hash_table) by using "append" command.
8. Return the candidate pairs from the bucket if the length of value in the bucket>1.
9. Hash_table is reset for every band, candidate pairs are kept in memory, later candidate pairs are made into a set so that they do no repeat.
9. For the candidate pairs, calculate the similarity based on their signature and the Jaccard similarity based on the hashed shingles.
10. Calculate the implied threshold(t) by using the function t = (1/b)^1/r. The implied threshold should be similar to the desired threshold and controls false-positives/negatives

In [6]:
def find_sim_LSH(b,signature):
    start_time=time.time()
    r=int(numH/b)
    candidates = []
    for band in range(0,b):
        hash_table = defaultdict(list)
        for doc_id in range(len(signature)):
            start=band*r
            end=(band+1)*r
            current_band=signature[doc_id][start:end]
            current_hash=hash(tuple(current_band))
            hash_table[current_hash].append(doc_id)
        for value in hash_table.values():
            if len(value) > 1:
                candidates += list(itertools.combinations(value, 2))  
    candidates=set(candidates)
    for pair in candidates:
        sign1=signature[pair[0]]
        sign2=signature[pair[1]]
        count=0
        for k in range(numH):
            count = count + (sign1[k] == sign2[k])
        print("Similarity of signature between {} and {} = {}. Jaccard={}".
              format(docNames[pair[0]],docNames[pair[1]],count/numH,
                     jacard_similarity(hash_shingle_sets[pair[0]], hash_shingle_sets[pair[1]])))   
    print("Time taken: %.8s seconds ---" % (time.time() - start_time))
    print('Implied threshold(1/b)^(1/r)={}'.format(pow((1/b),(1/r))))
    Implied_threshold=pow((1/b),(1/r))
    return Implied_threshold

#### 7. Look for the cadidate pairs with a given similarity threshold

1. Find the cadidate pairs by calculate the similarity between the signatures and returns similar pairs if similarity is more than the given threshold.
2. Inputs: threshold (thresh), signature_matrix (signature).
3. This function records the time taken to find all similar pairs.

In [7]:
# Similarity of signatures
def find_sim_all(thresh,signature):
    start_time = time.time()
    for i in range(len(docNames)):
        sig1 = signature[i]
        for j in range(i+1,len(docNames)):
            sig2=signature[j]
            count = 0
        # Count the number of positions in the minhash signature which are equal.
            for k in range(0, numH):
                count = count + (sig1[k] == sig2[k])
            if (i!=j) and count/numH>thresh:

                print("Similarity of signature between {} and {} = {}. Jaccard={}".
                      format(docNames[i],docNames[j],count/numH,
                             jacard_similarity(hash_shingle_sets[i], hash_shingle_sets[j])))
    print("Time taken: %.8s seconds ---" % (time.time() - start_time))

#### 8. Define file paths and some parameters which will be used in the implementation

In [8]:
corpus_folder = "./data/" 
# "./Untitled Folder/" here is the file path we want to deal with. 
docNames=os.listdir("data") 
# "./Untitled Folder/" here is the file path we want to deal with. 
numH=100
coefA=random.sample(range(2**32-1),numH)
coefB=random.sample(range(2**32-1),numH)
nextPrime=4294967311
file_list = glob.glob(os.path.join(corpus_folder, "*"))

In [9]:
file_list

['./data/cbchealth.txt',
 './data/cbcandreuters.txt',
 './data/cnnhealth.txt',
 './data/foxnewshealth.txt',
 './data/everydayhealth.txt',
 './data/nytimeshealth.txt',
 './data/bbchealth.txt',
 './data/reuters_health.txt',
 './data/msnhealthnews.txt']

#### 9. Call the hash_shingle function and transfer the documents in the file_list to hashed shingle sets.

In [10]:
hash_shingle_sets=list(map(hash_shingle,file_list))

#### 10. Calculate the Jaccard similarity of shinglings.

In [11]:
for i in range(3):
    shingle1 = hash_shingle_sets[i]
    for j in range(i+1,3):
        shingle2=hash_shingle_sets[j]
        print("Jaccard similarity of shinglings between {} and {} = {} ".
              format(docNames[i],docNames[j],jacard_similarity(shingle1, shingle2)))

Jaccard similarity of shinglings between cbchealth.txt and cbcandreuters.txt = 0.5286752042679373 
Jaccard similarity of shinglings between cbchealth.txt and cnnhealth.txt = 0.24989234838698338 
Jaccard similarity of shinglings between cbcandreuters.txt and cnnhealth.txt = 0.24020114840044224 


#### 9. Call the get_signature function and transfer the hashed shingle sets to signature matrix.

In [12]:
signature_matrix = list(map(get_signature , hash_shingle_sets))

#### 10.Call the find_sim_LSH function and return the candidate pairs, as well as their similarity.

In [13]:
find_sim_LSH(20,signature_matrix)

Time taken: 0.000267 seconds ---
Implied threshold(1/b)^(1/r)=0.5492802716530588


0.5492802716530588

#### 11. Find the cadidate pairs with a given similarity threshold by calculating the similarity between the signatures

In [14]:
find_sim_all(0.5,signature_matrix)

Similarity of signature between cbchealth.txt and cbcandreuters.txt = 0.51. Jaccard=0.5286752042679373


IndexError: list index out of range

### Reference

1. https://mccormickml.com/2015/06/12/minhash-tutorial-with-python-code/
2. Rajaraman, Anand, and Jeffrey David Ullman. Mining of massive datasets. Cambridge University Press, 2011.